# Topic modelling on news data 

- Data is taken from kaggle you can download it from below .
- Load the data using pandas module

In [1]:
import pandas as pd
data=pd.read_csv('abcnews-date-text.csv',error_bad_lines=False)

In [2]:
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [3]:
print(len(documents))
print(documents[:5])

1103665
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


# Data preprocessing 
## Different process we do here like as Tokenization ,lemetization and stemming the data
- we wanted to convert the data to their normal form for example 'stolen' would converted to steal
### Here nltk is used for removing different language rather than english (Hindi and urdu like that word would be removed)
- In preprocessing actually we remove all the punctuation marks , exclamatory marks and commas 

In [4]:

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
               
            

C:\Users\praveen\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\praveen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Lemmatization 
## Lemmatization is used for gouping of word that's contains the same meaning(synonyms,antonyms)
# Tokenization
## Tokenization is used for keeps the word having meaningfull meaning
- This is used for removal of word like if,the ,a,an that word doesn't make any sense in Topic
# Stemming
## Stemming is used for convert the word into their root form

In [5]:
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

# preview data after preprocessing 
- How the data will look like 

In [6]:
# Select a document to preview after preprocessing
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))



original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfires']


In [7]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0              [decide, community, broadcast, licence]
1                         [witness, aware, defamation]
2           [call, infrastructure, protection, summit]
3                          [staff, aust, strike, rise]
4             [strike, affect, australian, travellers]
5               [ambitious, olsson, win, triple, jump]
6               [antic, delight, record, break, barca]
7    [aussie, qualifier, stosur, waste, memphis, ma...
8             [aust, address, security, council, iraq]
9                         [australia, lock, timetable]
Name: headline_text, dtype: object

### Dictionary is formed for 

In [8]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 community
2 decide
3 licence
4 aware
5 defamation
6 witness
7 call
8 infrastructure
9 protection
10 summit


In [9]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [10]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(112, 1), (489, 1), (970, 1), (4427, 1)]

In [11]:
import pickle
pickle.dump(bow_corpus, open('bow_corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [12]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 112 ("help") appears 1 time.
Word 489 ("rain") appears 1 time.
Word 970 ("bushfires") appears 1 time.
Word 4427 ("dampen") appears 1 time.


In [13]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5955574839932255),
 (1, 0.394834368882524),
 (2, 0.4976240783705052),
 (3, 0.4917188028077664)]


In [14]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)
lda_model.save('model5.gensim')

In [15]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.028*"police" + 0.017*"charge" + 0.016*"court" + 0.012*"attack" + 0.012*"south" + 0.011*"murder" + 0.011*"kill" + 0.011*"death" + 0.010*"face" + 0.010*"woman"
Topic: 1 
Words: 0.010*"say" + 0.009*"country" + 0.009*"health" + 0.008*"fund" + 0.008*"rural" + 0.007*"indigenous" + 0.007*"tasmania" + 0.007*"turnbull" + 0.006*"australia" + 0.006*"fall"
Topic: 2 
Words: 0.017*"trump" + 0.011*"australia" + 0.010*"crash" + 0.009*"world" + 0.008*"live" + 0.008*"test" + 0.008*"donald" + 0.007*"win" + 0.006*"open" + 0.006*"deal"
Topic: 3 
Words: 0.019*"australian" + 0.011*"house" + 0.009*"market" + 0.008*"miss" + 0.007*"rise" + 0.007*"perth" + 0.007*"west" + 0.007*"high" + 0.007*"price" + 0.007*"break"
Topic: 4 
Words: 0.015*"government" + 0.013*"queensland" + 0.010*"warn" + 0.009*"plan" + 0.009*"say" + 0.008*"north" + 0.008*"election" + 0.007*"water" + 0.006*"coast" + 0.006*"gold"


In [16]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
lda_model.save('model10.gensim')
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))


Topic: 0 Word: 0.009*"donald" + 0.009*"weather" + 0.008*"sport" + 0.006*"friday" + 0.006*"rugby" + 0.006*"thursday" + 0.005*"dairy" + 0.004*"australia" + 0.004*"grand" + 0.004*"australian"
Topic: 1 Word: 0.020*"trump" + 0.009*"turnbull" + 0.007*"victorian" + 0.006*"tuesday" + 0.006*"violence" + 0.006*"asylum" + 0.006*"domestic" + 0.005*"tony" + 0.005*"kid" + 0.005*"quiz"
Topic: 2 Word: 0.015*"crash" + 0.012*"police" + 0.008*"woman" + 0.008*"die" + 0.008*"miss" + 0.007*"search" + 0.006*"dead" + 0.006*"shoot" + 0.006*"climate" + 0.006*"driver"
Topic: 3 Word: 0.015*"market" + 0.010*"share" + 0.007*"price" + 0.007*"australian" + 0.007*"marriage" + 0.006*"monday" + 0.006*"wall" + 0.006*"queensland" + 0.006*"fall" + 0.006*"hobart"
Topic: 4 Word: 0.013*"rural" + 0.010*"government" + 0.008*"health" + 0.008*"fund" + 0.006*"budget" + 0.006*"federal" + 0.005*"national" + 0.005*"plan" + 0.005*"election" + 0.005*"labor"
Topic: 5 Word: 0.009*"grandstand" + 0.007*"kill" + 0.007*"australia" + 0.007*"n

In [17]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfires']

In [18]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.34887999296188354	 
Topic: 0.010*"say" + 0.009*"country" + 0.009*"health" + 0.008*"fund" + 0.008*"rural" + 0.007*"indigenous" + 0.007*"tasmania" + 0.007*"turnbull" + 0.006*"australia" + 0.006*"fall"

Score: 0.3300514817237854	 
Topic: 0.015*"government" + 0.013*"queensland" + 0.010*"warn" + 0.009*"plan" + 0.009*"say" + 0.008*"north" + 0.008*"election" + 0.007*"water" + 0.006*"coast" + 0.006*"gold"

Score: 0.23995959758758545	 
Topic: 0.028*"police" + 0.017*"charge" + 0.016*"court" + 0.012*"attack" + 0.012*"south" + 0.011*"murder" + 0.011*"kill" + 0.011*"death" + 0.010*"face" + 0.010*"woman"

Score: 0.04072026163339615	 
Topic: 0.019*"australian" + 0.011*"house" + 0.009*"market" + 0.008*"miss" + 0.007*"rise" + 0.007*"perth" + 0.007*"west" + 0.007*"high" + 0.007*"price" + 0.007*"break"

Score: 0.04038867726922035	 
Topic: 0.017*"trump" + 0.011*"australia" + 0.010*"crash" + 0.009*"world" + 0.008*"live" + 0.008*"test" + 0.008*"donald" + 0.007*"win" + 0.006*"open" + 0.006*"deal"


In [19]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8199707865715027	 
Topic: 0.020*"trump" + 0.009*"turnbull" + 0.007*"victorian" + 0.006*"tuesday" + 0.006*"violence" + 0.006*"asylum" + 0.006*"domestic" + 0.005*"tony" + 0.005*"kid" + 0.005*"quiz"

Score: 0.02001131884753704	 
Topic: 0.009*"donald" + 0.009*"weather" + 0.008*"sport" + 0.006*"friday" + 0.006*"rugby" + 0.006*"thursday" + 0.005*"dairy" + 0.004*"australia" + 0.004*"grand" + 0.004*"australian"

Score: 0.02000722475349903	 
Topic: 0.015*"market" + 0.010*"share" + 0.007*"price" + 0.007*"australian" + 0.007*"marriage" + 0.006*"monday" + 0.006*"wall" + 0.006*"queensland" + 0.006*"fall" + 0.006*"hobart"

Score: 0.02000321075320244	 
Topic: 0.013*"rural" + 0.010*"government" + 0.008*"health" + 0.008*"fund" + 0.006*"budget" + 0.006*"federal" + 0.005*"national" + 0.005*"plan" + 0.005*"election" + 0.005*"labor"

Score: 0.020001837983727455	 
Topic: 0.015*"crash" + 0.012*"police" + 0.008*"woman" + 0.008*"die" + 0.008*"miss" + 0.007*"search" + 0.006*"dead" + 0.006*"shoot" + 0.

In [20]:
unseen_document = 'How a Pentagon de to help him identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.36665859818458557	 Topic: 0.028*"police" + 0.017*"charge" + 0.016*"court" + 0.012*"attack" + 0.012*"south"
Score: 0.2905018627643585	 Topic: 0.010*"say" + 0.009*"country" + 0.009*"health" + 0.008*"fund" + 0.008*"rural"
Score: 0.27556732296943665	 Topic: 0.015*"government" + 0.013*"queensland" + 0.010*"warn" + 0.009*"plan" + 0.009*"say"
Score: 0.03393510356545448	 Topic: 0.019*"australian" + 0.011*"house" + 0.009*"market" + 0.008*"miss" + 0.007*"rise"
Score: 0.03333709388971329	 Topic: 0.017*"trump" + 0.011*"australia" + 0.010*"crash" + 0.009*"world" + 0.008*"live"


In [21]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('bow_corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')


In [22]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\praveen\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [24]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus1 = pickle.load(open('bow_corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model10.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus1, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)